In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Trainer (has labels)
trainer_df = pd.read_csv('datasets/cleaned_SOLIDtest6K_trainer.tsv', sep="\t")
learner_tweets_df = pd.read_csv('datasets/cleaned_SOLID9M_learner.tsv', sep="\t")

In [2]:
import numpy as np
# trainer_labels = trainer_df['label'].values
# trainer_tweets = trainer_df['tweet'].values
learner_tweets_df['labels'] = learner_tweets_df['average'].apply(lambda x: 1 if x >= 0.5 else 0) # threshold the average values

sample_size = 80000
positive_ratio = 0.75

# Select the most confident positive values
semi_tweets_pos_df = learner_tweets_df[learner_tweets_df['average'] > 0.5].sample(n=np.floor(sample_size*positive_ratio).astype(int), random_state=1)

# Select the most confident negative values
semi_tweets_neg_df = learner_tweets_df[learner_tweets_df['average'] < 0.2].sample(n=np.floor(sample_size*(1-positive_ratio)).astype(int), random_state=1)

semi_tweets_df = pd.concat([semi_tweets_pos_df, semi_tweets_neg_df])
semi_tweets_df = semi_tweets_df.sample(frac=1, random_state=42)

semi_tweets = semi_tweets_df['text'].values
semi_labels = semi_tweets_df['labels'].values


In [3]:
semi_tweets_df.head(10)

,id,text,average,std,labels
6863712,1188439156923535360,you need to be in prison a coup in the us is t...,0.659527,0.132832,1
8272013,1186311236968947713,i do not fw her because she nasty but i fw her...,0.770702,0.137683,1
7847151,1188696905586528256,my eye was fascinated by that memorandum,0.196262,0.158945,0
2782796,1161519296075505665,oh well thats good sleep well then,0.182599,0.193224,0
6897871,1188503634515877888,i matheus would have consensual and hot sex wi...,0.501483,0.153216,1
3605572,1161423107808256001,as long as this guy keeps running his mouth an...,0.561974,0.139245,1
2190200,1159412408521318400,and you know the ish is staged and not genuine...,0.188691,0.168918,0
1595615,1161920544314294272,adnthweeksary for us to regard others as wo...,0.153824,0.174742,0
7611207,1187283571687473152,i could eat a horse hooves and all,0.511584,0.116744,1
7510725,1188619872294408197,max is a poor loser and the ppc have zero seat...,0.720084,0.143393,1


In [4]:
import torch
from torch.utils.data import DataLoader, Dataset


class TweetDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
learner_dataset = TweetDataset(semi_tweets, semi_labels)

# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
learner_loader = DataLoader(learner_dataset, batch_size=6, shuffle=False)

In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch import optim
from tqdm import tqdm

num_epochs = 1

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


model = BertForSequenceClassification.from_pretrained('models/OLID_BERT_1', num_labels=2)
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


/home/cymn/miniconda3/envs/nvidia-newest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
model.train()
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    semi_labeled_dataloader_iterator = iter(learner_loader)
    num_batches = len(learner_loader)
    # num_batches = 5000
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(semi_labeled_dataloader_iterator, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

  0%|          | 0/13334 [00:00<?, ?it/s]/tmp/ipykernel_3662/1995336368.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████| 13334/13334 [11:31<00:00, 19.29it/s]


In [ ]:
model.save_pretrained('models/SOLID_BERT_colearning_1')